<a href="https://colab.research.google.com/github/MortezaMahdaviMortazavi/ImageMatching-Hexathlon/blob/master/Image_Matching_Challenge_2024_Hexathlon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Dataset from kaggle

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()  # Upload the kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c image-matching-challenge-2024
!unzip -q image-matching-challenge-2024.zip

# Import packages

In [23]:
!pip install numpy pandas torch torchvision opencv-python scikit-learn matplotlib tqdm pycolmap pytransform3d
import torch
import torch.nn as nn
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import cv2 as cv
import torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 83.2 MB/s eta 0:00:00
  Created wheel for pytransform3d: filename=pytransform3d-3.7.0-py3-none-any.whl size=131540 sha256=3188e14dbdc525212bab9b1e59fd461bf8d0530c3fb47e543299a524c6c728ed
  Stored in directory: /root/.cache/pip/wheels/18/01/17/ba4ab8a4ec908b66758719239818d257c3b7b1a556f1239141
Successfully built pytransform3d


# Dependencies

In [38]:
from dataclasses import dataclass

@dataclass
class Config:
  ### Paths
  train_labels_path : str = "train/train_labels.csv"
  categories_path : str = "train/categories.csv"
  church_path : str = "train/church"
  dioscuri_path : str = "train/dioscuri"
  lizard_path : str = "train/lizard"
  pond_path : str = "train/pond"
  baalshamin_path : str = "train/multi-temporal-temple-baalshamin"
  cameras_path : str = "{INPUT_PATH}/sfm/cameras.txt"
  images_path : str = "{INPUT_PATH}/sfm/images.txt"
  points3d_path : str = "{INPUT_PATH}/sfm/points3D"

  ### Hyperparameters







# Load dataset and plot some of them

In [43]:
## Read cameras.txt file and convert it to dataframe for analyze it easily
def convert_cameras_txt_to_dataframe(txt_path):
    """
    Reads a cameras.txt file and converts it to a clean pandas DataFrame.
    """
    data = []
    with open(txt_path, 'r') as file:
        for line in file:
            if line.startswith("#") or line.strip() == "":
                continue  # Skip comments and empty lines
            line_parts = line.strip().split()
            camera_id = int(line_parts[0])
            model = line_parts[1]
            width = int(line_parts[2])
            height = int(line_parts[3])
            params = [float(param) for param in line_parts[4:]]  # Convert parameters to floats
            data.append([camera_id, model, width, height] + params)

    columns = ['CAMERA_ID', 'MODEL', 'WIDTH', 'HEIGHT', 'PARAM_1', 'PARAM_2', 'PARAM_3']
    df = pd.DataFrame(data, columns=columns)
    return df

In [46]:

def convert_points3d_txt_to_dataframe(txt_path):
    """
    Reads a 3D point list from a .txt file and converts it to a pandas DataFrame.
    The DataFrame contains 3D coordinates, color, error, and tracking information for each point.
    """
    data = []

    with open(txt_path, 'r') as file:
        for line in file:
            if line.startswith("#") or line.strip() == "":
                continue  # Skip comments and empty lines
            line_parts = line.strip().split()
            point_id = int(line_parts[0])
            x, y, z = map(float, line_parts[1:4])  # 3D coordinates
            r, g, b = map(int, line_parts[4:7])    # RGB color
            error = float(line_parts[7])           # Error
            tracks = [(int(line_parts[i]), int(line_parts[i + 1])) for i in range(8, len(line_parts), 2)]
            data.append([point_id, x, y, z, r, g, b, error, tracks])

    columns = ['POINT3D_ID', 'X', 'Y', 'Z', 'R', 'G', 'B', 'ERROR', 'TRACKS']
    df = pd.DataFrame(data, columns=columns)
    return df

In [50]:
def clean_images_txt_data_to_dataframe(txt_path):
    """
    Reads an image list with poses and 2D points from a .txt file and converts it to a pandas DataFrame.
    Each row in the DataFrame contains image pose attributes and corresponding 2D points.
    """
    data = []

    with open(txt_path, 'r') as file:
        lines = file.readlines()

        i = 0
        while i < len(lines):
            # Skip comment lines and empty lines
            if lines[i].startswith("#") or lines[i].strip() == "":
                i += 1
                continue

            # First line contains image metadata
            image_line = lines[i].strip().split()
            image_id = int(image_line[0])
            qw, qx, qy, qz = map(float, image_line[1:5])  # Quaternion
            tx, ty, tz = map(float, image_line[5:8])      # Translation
            camera_id = int(image_line[8])
            name = image_line[9]

            # Second line contains 2D points information
            points2d_line = lines[i + 1].strip().split()
            points2d = []
            for j in range(0, len(points2d_line), 3):
                x = float(points2d_line[j])
                y = float(points2d_line[j + 1])
                point3d_id = int(points2d_line[j + 2])
                points2d.append((x, y, point3d_id))

            # Append the data to the list
            data.append([image_id, qw, qx, qy, qz, tx, ty, tz, camera_id, name, points2d])

            # Move to the next image block (2 lines per image)
            i += 2

    columns = ['IMAGE_ID', 'QW', 'QX', 'QY', 'QZ', 'TX', 'TY', 'TZ', 'CAMERA_ID', 'NAME', 'POINTS2D']
    df = pd.DataFrame(data, columns=columns)

    return df

In [51]:
df = clean_images_txt_data_to_dataframe("train/church/sfm/images.txt")
df.head()

,IMAGE_ID,QW,QX,QY,QZ,TX,TY,TZ,CAMERA_ID,NAME,POINTS2D
0,111,0.700102,-0.028844,0.639510,-0.316310,-0.441407,2.700312,4.340615,111,00111.jpg,"[(266.75, 565.5, 15247), (457.75, 507.0, 40011..."
1,110,0.709290,-0.029780,0.626371,-0.321994,-0.568954,2.771873,4.318070,110,00110.jpg,"[(367.5, 873.0, -1), (430.25, 938.5, 39788), (..."
2,109,0.753859,-0.017284,0.587321,-0.294029,-0.819819,2.113128,3.545903,109,00109.jpg,"[(313.0, 313.25, 1), (517.5, 744.0, 39651), (2..."
3,108,0.845267,-0.000042,0.493965,-0.203769,-1.614110,1.790496,3.732522,108,00108.jpg,"[(268.5, 391.0, 18587), (383.75, 655.0, 37885)..."
4,107,0.850515,-0.029079,0.474176,-0.225687,-1.223845,1.943315,3.304654,107,00107.jpg,"[(243.0, 356.25, 2), (557.0, 290.0, -1), (545...."
